In [20]:
import pandas as pd
import numpy as np 
import multiprocessing as mp
from multiprocessing import cpu_count
from concurrent.futures import ProcessPoolExecutor
import os
import datetime
import time
import sys
from typing import List



读取数据（取最小行数）

In [21]:
datalist = []
min_rows = float('inf') 

for filename in os.listdir("/home/yichuan/ywc/meta-labeling/cryptocurrency"):
    if filename.endswith('.csv'):
        file_path = os.path.join("/home/yichuan/ywc/meta-labeling/cryptocurrency", filename)
        df = pd.read_csv(file_path)
        df = df.drop(columns=["Date"])
        datalist.append(df.values)
        min_rows = min(min_rows, df.shape[0])      

data_list = [data[:min_rows] for data in datalist]

print(min_rows,len(data_list))

1878 4


处理最后一列

In [22]:
def logVolume(X: np.ndarray):
    volume = X[:, -1]  
    volume_log = np.log(volume + 1)  

    mean_log = np.mean(volume_log)  
    std_log = np.std(volume_log)  

    volume_std = (volume_log - mean_log) / std_log  

    X[:, -1] = volume_std  

    return X


for data in data_list:
    data=logVolume(data)    

print(data_list[1])

[[ 0.21791101  0.221791    0.214866    0.21748801  0.21748801 -2.28482287]
 [ 0.218256    0.21906801  0.20525999  0.20648301  0.20648301 -2.33154956]
 [ 0.205948    0.21445601  0.205459    0.21043     0.21043    -2.37803102]
 ...
 [ 0.36812201  0.368543    0.35613599  0.35653099  0.35653099 -0.58243597]
 [ 0.35652399  0.357225    0.338911    0.34284601  0.34284601 -0.51650078]
 [ 0.34283099  0.34531301  0.33533299  0.34501699  0.34501699 -0.80060086]]


创建标签函数

In [23]:

days=10

ptsl = [0.05, 0.05]  
return_min = 0.005
def triple_barrier(close: np.ndarray, days=10, pts=[0.05, 0.05] ):
    bin = np.zeros(close.size, dtype=int)  
    for i in range(close.size):
        for d in range(days):
            index = min(i + d + 1, close.size - 1)  
            if close[index] >= close[i] * (1 + ptsl[0]):  
                bin[i] = 1
                break
            elif close[index] <= close[i] * (1 - ptsl[1]):  
                bin[i] = -1
                break
    
    return bin




In [24]:
binmat=np.full((min_rows, 4), 3)
for i in range(4):
    binmat[:,i]=triple_barrier(data_list[i][:,3])

label=np.mean(binmat, axis=1)
print(label.shape)

(1878,)


合并

In [25]:
# 确保所有二维数组具有相同的形状
shapes = {data.shape for data in data_list}
if len(shapes) > 1:
    raise ValueError("所有 CSV 文件中的数据形状必须一致")
    
dataset=np.array(data_list)
print(dataset.shape)
dataset = dataset.transpose(1, 0, 2).reshape(1878, -1)
print(dataset.shape)

(4, 1878, 6)
(1878, 24)


Primary model with constraints

In [26]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import xgboost as xgb

In [27]:
targets=(label + 1)/2
targets=np.where(targets > 0.5, 1, 0)
indices = np.arange(len(dataset))
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(dataset,targets,indices,test_size=0.2,random_state=42)

In [28]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [29]:
def asset_exposure_constraint(weights, max_exposure=0.5):
    """Ensure no asset weight exceeds max_exposure."""
    return all(abs(w) <= max_exposure for w in weights)

def sector_diversification_constraint(weights, sector_mapping, min_sectors=2):
    """Ensure allocation spans at least min_sectors."""
    unique_sectors = set(sector_mapping[np.nonzero(weights)])
    return len(unique_sectors) >= min_sectors


def risk_tolerance_constraint(weights, cov_matrix, max_risk=0.05):
    """Ensure portfolio variance is within max_risk."""
    portfolio_variance = np.dot(weights.T, np.dot(cov_matrix, weights))
    #print(portfolio_variance)
    return portfolio_variance <= max_risk

def apply_constraints(weights, sector_mapping, cov_matrix):
    """Check all constraints."""
    return (
        asset_exposure_constraint(weights, max_exposure=0.5) and
        sector_diversification_constraint(weights, sector_mapping, min_sectors=2) and
        risk_tolerance_constraint(weights, cov_matrix, max_risk=0.05)
    )

In [30]:
# Primary model using XGBoost
model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    objective='binary:logistic',
    eval_metric='logloss'
)
model.fit(X_train, y_train)

y_pred_primary = model.predict(X_test)
primary_signals = np.where(y_pred_primary > 0.5, 1, 0)


In [31]:
print(primary_signals)
print(len(primary_signals))
print(X_test.shape)

[1 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 1 1
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 1 1 1 1 0
 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0 1 0 0
 0 0 1 0 0 1 0 1 0 0 1 0 0 0 1 0 0 1 1 1 0 0 1 1 0 0 0 0 1 1 1 0 0 0 0 1 0
 1 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 1 1 1 0 1 0 0 1 0 0 0 0 0 1 1 0 0 0 0
 0 0 0 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 1 0 1 0 1 1 0 0 0
 0 0 0 0 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 1 0 0 0 1 1 0
 0 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1 1 1 1 0 0 0 0 0 1 0 0 0 1 0 1
 0 1 1 0 1 0]
376
(376, 24)


In [32]:
print(X_train.shape)
print(X_test.shape)
print(y_test.shape)

(1502, 24)
(376, 24)
(376,)


In [33]:
# Assume asset returns, sector mapping, and covariance matrix as placeholders
sector_mapping = np.array([1,2,3,4])

Closelist=[]
for filename in os.listdir("/home/yichuan/ywc/meta-labeling/cryptocurrency"):
    if filename.endswith('.csv'):
        file_path = os.path.join("/home/yichuan/ywc/meta-labeling/cryptocurrency", filename)
        df = pd.read_csv(file_path)
        df = df["Adj Close"]
        Closelist.append(df.values)
        min_rows = min(min_rows, df.shape[0])
Closelist = [data[:min_rows] for data in Closelist]
Close=np.array(Closelist).T
Close_test=Close[sorted(indices_test),:]
print(Close_test.shape)
returns = (Close_test[1:] - Close_test[:-1]) / Close_test[:-1]  
cov_matrix = np.cov(returns, rowvar=False)
print(cov_matrix)

asset_returns = np.random.normal(0.01, 0.02, size=dataset.shape[1])



(376, 4)
[[0.00975429 0.00216908 0.00084753 0.01012304]
 [0.00216908 0.06281248 0.01614855 0.00204031]
 [0.00084753 0.01614855 0.01446641 0.00049159]
 [0.01012304 0.00204031 0.00049159 0.02662378]]


Secondary model (meta-labeling)

In [34]:
# Step 3: Develop secondary model (meta-labeling) 二级模型，即meta-labeling
# Create meta-labels for secondary model training
meta_labels = primary_signals == y_test
t_indices = np.arange(len(X_test))
X_train_meta, X_test_meta, y_train_meta, y_test_meta, t_indices_train, t_indices_test = train_test_split(
    X_test, meta_labels, t_indices, test_size=0.5, random_state=42
)
X_test_meta = pd.DataFrame(X_test_meta)


# Train and test secondary model (meta-labeling) 训练模型并生成二级信号
logistic = LogisticRegression()
logistic.fit(X_train_meta, y_train_meta)
y_pred_secondary = logistic.predict(X_test_meta)


# Step 4: Filtering, denoising, and refining signals 进行信号过滤，生成最终信号
refined_signals = primary_signals.copy()
zero_pred_mask = (y_pred_secondary == 0)
meta_zero_indices = np.where(zero_pred_mask)[0]  
original_test_indices = t_indices_test[meta_zero_indices]  
#print(original_test_indices)
for i in original_test_indices:
    refined_signals[i] = (1-refined_signals[i])






In [39]:
print(len(refined_signals),np.sum(abs(refined_signals-primary_signals)))
print(refined_signals.shape,refined_signals)

376 12
(376,) [0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 1 1
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 0 0 0 1 0 0 1 1 1 1 0
 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1
 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 0 1 1 1 0 0 1 1 0 0 0 0 1 1 1 0 0 0 0 1 0
 1 1 0 1 0 1 0 0 0 1 0 0 1 0 0 0 0 1 1 1 1 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0
 0 0 0 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 1 0 1 0 1 1 0 0 0
 0 0 1 0 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 1 0 0 0 1 1 0
 1 0 0 0 0 0 0 1 0 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 1 1 0 0 0 0 1 1 1 1 0 0 0 0 0 1 0 0 0 1 0 1
 0 1 1 0 1 0]


Generate modified(final) portfolio 

In [40]:
# Create a sample portfolio based on signals
def construct_portfolio(signals, asset_returns, sector_mapping, cov_matrix):
    """Construct a portfolio based on signals and constraints."""
    weights = np.zeros((len(signals),4))
    for i, signal in enumerate(signals):
        if signal == 1:
            # Placeholder: Assign random initial weights to activated signals
            weights[i,:] = np.random.rand(4)
            # Normalize weights
            weights[i,:] /= np.sum(weights[i,:])
            # Apply constraints
            if apply_constraints(weights[i,:], sector_mapping, cov_matrix):
                weights[i,:] = weights[i,:]
                #print("Constraints good.")
            else:
                #print(weights[i,:])
                weights[i,:] = np.zeros_like(weights[i,:]) #adjust weights
                #print("Constraints not satisfied. Adjust weights.")

    return weights   

In [41]:
final_portfolio_weights = construct_portfolio(refined_signals, asset_returns, sector_mapping, cov_matrix)
tradedays=0
# Output portfolio weights
print("Final Portfolio Weights:\n", final_portfolio_weights)
for i in range(final_portfolio_weights.shape[0]):
    if np.sum(final_portfolio_weights[i]) !=0:
        tradedays+=1
print("active trade days:",tradedays,final_portfolio_weights.shape)

Final Portfolio Weights:
 [[0.         0.         0.         0.        ]
 [0.34400156 0.18495151 0.38604635 0.08500058]
 [0.         0.         0.         0.        ]
 ...
 [0.         0.         0.         0.        ]
 [0.20318597 0.14334141 0.34163102 0.3118416 ]
 [0.         0.         0.         0.        ]]
active trade days: 102 (376, 4)


Evaluate the performance of the portfolio

In [38]:

def calculate_sharpe_ratio(stock_files):

    # 读取并合并数据
    dfs = []
    for i, file in enumerate(stock_files, 1):
        df = pd.read_csv(file)
        # 规范列名：假设原始列名为 open, close, high, low, weight, date
        df = df.rename(columns={
            'open': f'stock{i}_open',
            'close': f'stock{i}_close',
            'high': f'stock{i}_high',
            'low': f'stock{i}_low',
            'weight': f'weight{i}',
            'date': 'date'  # 确保日期列名统一
        })
        dfs.append(df)
    
    # 按日期合并所有数据（假设日期已对齐）
    merged = dfs[0]
    for df in dfs[1:]:
        merged = merged.merge(df, on='date', how='inner')
    
    # 计算每个股票的日收益率
    for i in range(1, 5):
        open_col = f'stock{i}_open'
        close_col = f'stock{i}_close'
        merged[f'return{i}'] = (merged[close_col] - merged[open_col]) / merged[open_col]
    
    # 计算组合日收益率（包含现金部分）
    weights = merged[[f'weight{i}' for i in range(1, 5)]]
    returns = merged[[f'return{i}' for i in range(1, 5)]]
    merged['portfolio_return'] = (weights * returns).sum(axis=1)
    
    # 现金部分权重（总和≤1）
    cash_weight = 1 - weights.sum(axis=1)
    merged['portfolio_return'] += cash_weight * 0  # 明确体现现金部分
    
    # 计算夏普比率
    mu_daily = merged['portfolio_return'].mean()
    sigma_daily = merged['portfolio_return'].std()
    
    if sigma_daily == 0:
        return 0.0
    
    # 年化（按365天）
    sharpe_ratio = (mu_daily * 365) / (sigma_daily * np.sqrt(365))
    return sharpe_ratio

# 示例用法
if __name__ == "__main__":
    stock_files = [
        'path/to/stock1.csv',
        'path/to/stock2.csv',
        'path/to/stock3.csv',
        'path/to/stock4.csv'
    ]
    sharpe = calculate_sharpe_ratio(stock_files)
    print(f"年化夏普比率: {sharpe:.4f}")

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/stock1.csv'